In [32]:
import praw # Python Reddit API Wrapper
#Praw documentation: https://praw.readthedocs.io/en/stable/index.html

# Create a Reddit instance
# This is a read-only instance since we only need to read data
# We don't need it to be authenticated since we aren't posting anything
reddit = praw.Reddit(
    client_id='51f3IBDpHGRzaGh0FGj0ng',
    client_secret=	'C-mvx1K5Flj_4kM-Grc9FLT2_bHxag',
    user_agent='TradingAnalysisApp by /u/Ok_Ability_9624'
)


In [33]:
print(reddit.read_only)

True


In [34]:
# Fetch and print the titles of the top 10 hot posts in the 'test' subreddit
for submission in reddit.subreddit("test").hot(limit=10):
    print(submission.title)

Some test commands
Test me123
Test
just a test
zzasfda
My first text post with title "The great cookie caper"
Mikes number 1 enemey
Rage
please dont be mean to isaam :(
Test


In [35]:
subreddit = reddit.subreddit("wallstreetbets")
print(subreddit.display_name)  
print(subreddit.title)
print(subreddit.description)


wallstreetbets
wallstreetbets
The rules and submission guidelines are maintained on new Reddit so be sure to check them and make sure you're up to date.

* Read the [rules](https://www.reddit.com/r/wallstreetbets/about/rules)

* Read the [comment and submission guide](https://www.reddit.com/r/wallstreetbets/wiki/contentguide)

* Read the [FAQ](https://www.reddit.com/r/wallstreetbets/wiki/faq) if you're new to both wallstreetbets and trading.

---
**Join the discord**

[WSB Discord](https://discord.gg/wsbverse)

**filter by flairs**

^Navigate ^WSB |^We ^recommend ^best ^daily ^DD
:--|:--     
**DD** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restrict_sr=on&q=flair%3ADD) / [**Best Daily**](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=day) / [Best Weekly](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=week)
**Discussion** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restric

In [36]:

postData = []
for submission in subreddit.hot(limit=10000):
        postData.append({
        'title': submission.title,
        'score': submission.score,  # upvotes
        'id': submission.id,
        'url': submission.url,
        'comms_num': submission.num_comments,
        'created': submission.created_utc,
        'body': submission.selftext
        })

In [37]:
import pandas as pd

df = pd.DataFrame(postData)
df

,title,score,id,url,comms_num,created,body
0,"What Are Your Moves Tomorrow, October 22, 2024",67,1g8z4d2,https://www.reddit.com/r/wallstreetbets/commen...,1438,1.729541e+09,This post contains content not supported on ol...
1,Weekly Earnings Thread 10/21 -10/25,309,1g6yf0l,https://i.redd.it/5e4zyncicmvd1.jpeg,532,1.729303e+09,
2,Just put the fries in the bag,1633,1g8yz6v,https://i.redd.it/6213bf29z5wd1.jpeg,53,1.729540e+09,
3,HOW DO I TELL MY PARENTS / FIANCÉ I LOST ALL M...,1729,1g8tfk7,https://www.reddit.com/r/wallstreetbets/commen...,1034,1.729527e+09,"I’ll get a whole lecture if they find out, so ..."
4,NVDA put buyers today,397,1g8zu8l,https://i.redd.it/mn59i2ac56wd1.jpeg,25,1.729542e+09,
...,...,...,...,...,...,...,...
492,Betted on TSLA event failing and it worked,104,1g1hx64,https://i.redd.it/c8mqsz8sh6ud1.jpeg,9,1.728675e+09,
493,Optimus can talk,1632,1g11zlm,https://v.redd.it/j4g0ig4g42ud1,503,1.728622e+09,
494,Private Equity Taking Peloton Private? (PTON),24,1g0oewr,https://www.reddit.com/r/wallstreetbets/commen...,10,1.728582e+09,"Listen up regards,\n\n[https://13f.info/manage..."
495,Why is Warren Buffett hoarding such a huge cas...,4406,1fyzh4n,https://www.reddit.com/r/wallstreetbets/commen...,1227,1.728393e+09,https://preview.redd.it/u8w3e24o7jtd1.png?widt...
